# data preprocessing

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split


In [2]:
# Load the dataset
df = pd.read_csv(r"C:\Users\Microsoft\Downloads\anime.csv")
df.head()

anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266

In [3]:
# Handle missing values
df = df.dropna()

In [4]:
# Explore the dataset
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 12017 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12017 non-null  int64  
 1   name      12017 non-null  object 
 2   genre     12017 non-null  object 
 3   type      12017 non-null  object 
 4   episodes  12017 non-null  object 
 5   rating    12017 non-null  float64
 6   members   12017 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 751.1+ KB


In [5]:
df.describe()

anime_id        rating       members
count  12017.000000  12017.000000  1.201700e+04
mean   13638.001165      6.478264  1.834888e+04
std    11231.076675      1.023857  5.537250e+04
min        1.000000      1.670000  1.200000e+01
25%     3391.000000      5.890000  2.250000e+02
50%     9959.000000      6.570000  1.552000e+03
75%    23729.000000      7.180000  9.588000e+03
max    34519.000000     10.000000  1.013917e+06

# Features extraction

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

def extract_features(df):
    # Genre encoding
    genre_vectorizer = CountVectorizer(tokenizer=lambda x: x.split(','))
    genre_encoded = genre_vectorizer.fit_transform(df['genre'])
    genre_df = pd.DataFrame(genre_encoded.toarray(), columns=genre_vectorizer.get_feature_names_out())
    
    # Normalize ratings
    scaler = MinMaxScaler()
    df['normalized_rating'] = scaler.fit_transform(df[['rating']])
    
    # Encode broadcast type (if available)
    if 'type' in df.columns:
        type_dummies = pd.get_dummies(df['type'], prefix='type')
    
    # Handle 'episodes' column
    if 'episodes' in df.columns:
        # Convert 'Unknown' to NaN and then to numeric
        df['episodes'] = pd.to_numeric(df['episodes'].replace('Unknown', np.nan), errors='coerce')
        # Fill NaN with median
        df['episodes'] = df['episodes'].fillna(df['episodes'].median())
        df['normalized_episodes'] = scaler.fit_transform(df[['episodes']])
    
    # Handle 'members' column
    if 'members' in df.columns:
        # Convert to numeric, replacing any non-numeric values with NaN
        df['members'] = pd.to_numeric(df['members'], errors='coerce')
        # Fill NaN with median
        df['members'] = df['members'].fillna(df['members'].median())
        df['normalized_members'] = scaler.fit_transform(df[['members']])
    
    # Combine all features
    feature_df = pd.concat([
        genre_df, 
        df['normalized_rating'],
        type_dummies if 'type' in df.columns else pd.DataFrame(),
        df['normalized_episodes'] if 'episodes' in df.columns else pd.Series(),
        df['normalized_members'] if 'members' in df.columns else pd.Series()
    ], axis=1)
    
    return feature_df

# Assuming you have your dataframe loaded as 'df'
features = extract_features(df)

# Print the first few rows and shape of the features DataFrame
print(features.head())
print(f"Shape of features DataFrame: {features.shape}")

    adventure   cars   comedy   dementia   demons   drama   ecchi   fantasy  \
0         0.0    0.0      0.0        0.0      0.0     0.0     0.0       0.0   
1         1.0    0.0      0.0        0.0      0.0     1.0     0.0       1.0   
2         0.0    0.0      1.0        0.0      0.0     0.0     0.0       0.0   
3         0.0    0.0      0.0        0.0      0.0     0.0     0.0       0.0   
4         0.0    0.0      1.0        0.0      0.0     0.0     0.0       0.0   

    game   harem  ...  yaoi  normalized_rating  type_Movie  type_Music  \
0    0.0     0.0  ...   0.0           0.924370        True       False   
1    0.0     0.0  ...   0.0           0.911164       False       False   
2    0.0     0.0  ...   0.0           0.909964       False       False   
3    0.0     0.0  ...   0.0           0.900360       False       False   
4    0.0     0.0  ...   0.0           0.899160       False       False   

   type_ONA  type_OVA  type_Special  type_TV  normalized_episodes  \
0     False

C:\Users\Microsoft\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# Recommendation system

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import cosine_similarity

def extract_features(df):
    # Genre encoding
    genre_vectorizer = CountVectorizer(tokenizer=lambda x: x.split(','))
    genre_encoded = genre_vectorizer.fit_transform(df['genre'].fillna(''))
    genre_df = pd.DataFrame(genre_encoded.toarray(), columns=genre_vectorizer.get_feature_names_out())
    
    # Normalize ratings
    scaler = MinMaxScaler()
    df['normalized_rating'] = scaler.fit_transform(df[['rating']].fillna(df['rating'].mean()))
    
    # Encode broadcast type (if available)
    if 'type' in df.columns:
        type_dummies = pd.get_dummies(df['type'], prefix='type')
    
    # Handle 'episodes' column
    if 'episodes' in df.columns:
        df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')
        df['normalized_episodes'] = scaler.fit_transform(df[['episodes']].fillna(df['episodes'].median()))
    
    # Handle 'members' column
    if 'members' in df.columns:
        df['members'] = pd.to_numeric(df['members'], errors='coerce')
        df['normalized_members'] = scaler.fit_transform(df[['members']].fillna(df['members'].median()))
    
    # Combine all features
    feature_df = pd.concat([
        genre_df, 
        df['normalized_rating'],
        type_dummies if 'type' in df.columns else pd.DataFrame(),
        df['normalized_episodes'] if 'episodes' in df.columns else pd.Series(),
        df['normalized_members'] if 'members' in df.columns else pd.Series()
    ], axis=1)
    
    # Handle any remaining NaN values
    imputer = SimpleImputer(strategy='mean')
    feature_matrix = imputer.fit_transform(feature_df)
    
    return pd.DataFrame(feature_matrix, columns=feature_df.columns)

# Extract features
features = extract_features(df)

def recommend_anime(target_anime, feature_matrix, df, n=5):
    # Calculate cosine similarity
    cosine_sim = cosine_similarity(feature_matrix)
    
    # Get the index of the target anime
    target_indices = df[df['name'] == target_anime].index
    if len(target_indices) == 0:
        print(f"Anime '{target_anime}' not found in the dataset.")
        return []
    
    idx = target_indices[0]
    
    # Get similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort anime based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get top N most similar anime
    sim_scores = sim_scores[1:n+1]
    anime_indices = [i[0] for i in sim_scores]
    
    return df['name'].iloc[anime_indices]

# Example usage
print(recommend_anime("Death Note", features, df))

C:\Users\Microsoft\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


144         Higurashi no Naku Koro ni Kai
778                    Death Note Rewrite
833              Jigoku Shoujo Mitsuganae
2691    Yakushiji Ryouko no Kaiki Jikenbo
6323              Saint Luminous Jogakuin
Name: name, dtype: object


# Evaluation

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Split the data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.shape,test_df.shape

((9613, 10), (2404, 10))

In [17]:
from sklearn.model_selection import train_test_split
import numpy as np

# Split the data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Extract features for train set
train_features = extract_features(train_df)

def evaluate_recommendations(true_genres, recommended_genres):
    true_set = set(true_genres.split(','))
    recommended_set = set(recommended_genres.split(','))
    
    intersection = len(true_set.intersection(recommended_set))
    precision = intersection / len(recommended_set) if recommended_set else 0
    recall = intersection / len(true_set) if true_set else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Evaluate on a sample of test set
sample_size = min(100, len(test_df))  # Adjust sample size as needed
test_sample = test_df.sample(n=sample_size, random_state=42)

results = []
for _, anime in test_sample.iterrows():
    recommendations = recommend_anime(anime['name'], train_features, train_df)
    recommended_genres = ','.join(train_df[train_df['name'].isin(recommendations)]['genre'].tolist())
    results.append(evaluate_recommendations(anime['genre'], recommended_genres))

precisions, recalls, f1_scores = zip(*results)

print(f"Average Precision: {np.mean(precisions)}")
print(f"Average Recall: {np.mean(recalls)}")
print(f"Average F1-score: {np.mean(f1_scores)}")

C:\Users\Microsoft\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Anime 'Ki Renka' not found in the dataset.
Anime 'Bakunyuu Shimai' not found in the dataset.
Anime 'La Vilaine LuLu' not found in the dataset.
Anime 'Girls und Panzer: Kore ga Hontou no Anzio-sen Desu!' not found in the dataset.
Anime 'Tokubetsu Jugyou' not found in the dataset.
Anime 'Cello Hiki no Gauche (OVA)' not found in the dataset.
Anime 'Ame-iro Cocoa in Hawaii' not found in the dataset.
Anime 'Chobits Recap' not found in the dataset.
Anime 'Bobby ni Kubittake' not found in the dataset.
Anime 'Kujibiki♥Unbalance' not found in the dataset.
Anime 'King of Prism by Pretty Rhythm Short Anime' not found in the dataset.
Anime 'Yuusha Shirei Dagwon: Suishou no Hitomi no Shounen' not found in the dataset.
Anime 'Zombie Clay Animation: Life of the Dead' not found in the dataset.
Anime 'Shin Seiki Cream Lemon' not found in the dataset.
Anime 'Shin Shirayuki-hime Densetsu Prétear' not found in the dataset.
Anime 'Bokura no Hero' not found in the dataset.
Anime 'Nanairo Kakumei' not found 

# 1can you explain the difference between user-based and item-based collaborative filtering?

ANS: User-based CF focuses on user similarity, while item-based CF focuses on item similarity.
User-based CF can be more personalized but may suffer from the cold-start problem for new users. Item-based CF is generally more stable and can handle new users better as long as they have some interaction history.

# 2What is collaborative filtering, and how does it work?

A.Collaborative filtering is a technique used in recommendation systems that makes automatic predictions about the interests of a user by collecting preferences or taste information from many users.

B.Collaborative filtering can work well when there's enough data, but it can suffer from the cold-start problem and sparsity issues in the user-item interaction matrix.